In [1]:
# import the usual
# import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
import numpy as np
# import hashlib
import datetime
#import surprise
#from surprise import SVD
%matplotlib inline
pd.set_option('display.max_columns', 500)

import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit # The Cython library


In [2]:
## create function to recommend using ALS for user_id
def ALS_recommend_for_user(model_given, user_id, sparse_user_item_field, N , orig_df ): #will add moe parameters later
    #user_id = 150
    ##### can be used for any model
    # Use the implicit recommender.
    #recommended = model.recommend(user_id, sparse_user_item, N, filter_already_liked_items = False)
    # got zero
    
    recommended = model_given.recommend(user_id, sparse_user_item_field, N )
    
    songs = []
    scores = []
    song_id_new_list = []
    
    # Get artist names from ids
    for item in recommended:
        idx, score = item
        #songs.append(user_song_df_count.song.loc[user_song_df_count.song_id_new == idx].iloc[0])
        #orig_df
        songs.append(user_song_df_count.song.loc[orig_df.song_id_new == idx].iloc[0])
        scores.append(score)
        #song_id_new_list.append(idx) #idx is not what it seems

    # Create a dataframe of artist names and scores
    recommendations = pd.DataFrame({'Song': songs, 'score': scores})
    #recommendations = pd.DataFrame({'Song': songs, 'song_id_new': song_id_new_list, 'score': scores})

    #print ("Top ", N, "recommendations for user_id:", user_id)
    return (recommendations)

## Read slot data here for user_song_df_count

In [6]:
slot1 = pd.read_csv(r"F:\Data_Repository\lastfm\df_slot1.tsv", sep="\t", error_bad_lines=False, header = None)
slot2 = pd.read_csv(r"F:\Data_Repository\lastfm\df_slot2.tsv", sep="\t", error_bad_lines=False, header = None)
slot3 = pd.read_csv(r"F:\Data_Repository\lastfm\df_slot3.tsv", sep="\t", error_bad_lines=False, header = None)
slot4 = pd.read_csv(r"F:\Data_Repository\lastfm\df_slot4.tsv", sep="\t", error_bad_lines=False, header = None)

In [7]:
slots = [slot1, slot2, slot3, slot4]
#loop through the slots

In [8]:
slotid = 1
for user_song_df_count in slots:
    print ("*** Slot no. = ", slotid)
    user_song_df_count.columns = ['userid', 'song', 'count']
    # Convert userid and song names into numerical IDs
    user_song_df_count['user_id'] = user_song_df_count['userid'].astype("category").cat.codes
    user_song_df_count['song_id_new'] = user_song_df_count['song'].astype("category").cat.codes

    # Create a lookup frame so we can get the artist names back in 
    # readable form later.
    item_lookup = user_song_df_count[['song_id_new', 'song']].drop_duplicates()
    item_lookup['song_id_new'] = item_lookup.song_id_new.astype(str)

    user_song_df = user_song_df_count.drop( ['userid', 'song'], axis = 1)

    M = 20000000 #slice of df

    altered_songid_new_list = user_song_df.loc[user_song_df[:M].groupby('user_id')['song_id_new'].head(1).index, 'song_id_new']
    user_song_df.loc[user_song_df[:M].groupby('user_id')['count'].head(1).index, 'count'] = np.NaN
    altered_user_id_list = user_song_df.loc[user_song_df[:M].groupby('user_id')['user_id'].head(1).index, 'user_id']

    altered_user_id_list = altered_user_id_list.reset_index(drop = True) 
    altered_songid_new_list = altered_songid_new_list.reset_index(drop = True) 
    print (len(altered_songid_new_list), "users changed" )
    user_song_df = user_song_df.dropna()     #drop all rows that have any NaN values
    #
    # Create lists of all users, songs and counts
    users = list(np.sort(user_song_df.user_id.unique()))
    songs = list(np.sort(user_song_df.song_id_new.unique()))
    counts = list(user_song_df['count'])


    sparse_item_user = sparse.csr_matrix((user_song_df['count'].astype(float), 
                                          (user_song_df['song_id_new'], user_song_df['user_id'])))

    sparse_user_item = sparse.csr_matrix((user_song_df['count'].astype(float), 
                                          (user_song_df['user_id'], user_song_df['song_id_new'])))

    ##ALS IMPLICIT
    print("")
    print ("###ALS IMPLICIT model")
    # Initialize the als model and fit it using the sparse item-user matrix
    model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

    # Calculate the confidence by multiplying it by our alpha value.
    alpha_val = 15
    data_conf = (sparse_item_user * alpha_val).astype('double')

    model.fit(data_conf)

    altered_songid_new_list = altered_songid_new_list.reset_index(drop = True) 


    cnt_total = []
    index1 = 0

    for idx in altered_user_id_list[:200]:
        #print(index1)
        recommendation_ans = ALS_recommend_for_user(model, idx, sparse_user_item, 30, user_song_df_count)
        ans = pd.Series(recommendation_ans['Song'].isin( 
                                            [ 
                                                item_lookup[ item_lookup['song_id_new'] == str(altered_songid_new_list[index1]) ]['song'].tolist()[0]
                                            ] ),
                        recommendation_ans['Song'].values)==1
        cnt = 0
        for i in ans:
            if i == True:
                cnt += 1 #increment if found true
        cnt_total.append ( cnt ) #append to total found
        index1 += 1

    N = len(recommendation_ans) #what N was set as, no. of items we recommended
    precision = []

    for i in range(len(cnt_total)): #loop 0 to N-1
        precision.append( cnt_total[i] / N )

    print("Precision at rank ", N)
    #precision
    print ( sum(precision) / len(precision) )

    ###Bayesian personalised ranking
    print("")
    print ( "####Bayesian Personalised Ranking model")

    modelBayesian = implicit.bpr.BayesianPersonalizedRanking(factors=20, regularization=0.1, iterations=20)
    # Calculate the confidence by multiplying it by our alpha value.
    alpha_val = 15
    data_conf = (sparse_item_user * alpha_val).astype('double')
    #fit model
    modelBayesian.fit(data_conf)

    
    cnt_total = []
    index1 = 0
    for idx in altered_user_id_list[:50]:
        #print(index1)
        recommendation_ans = ALS_recommend_for_user(modelBayesian, idx, sparse_user_item, 30, user_song_df_count)
        ans = pd.Series(recommendation_ans['Song'].isin( 
                                            [ 
                                                item_lookup[ item_lookup['song_id_new'] == str(altered_songid_new_list[index1]) ]['song'].tolist()[0]
                                            ] ),
                        recommendation_ans['Song'].values)==1
        cnt = 0
        for i in ans:
            if i == True:
                cnt += 1 #increment if found true
        cnt_total.append ( cnt ) #append to total found
        index1 += 1

    N = len(recommendation_ans) #what N was set as, no. of items we recommended
    precision = []

    for i in range(len(cnt_total)): #loop 0 to N-1
        precision.append( cnt_total[i] / N )

    print("Precision at rank ", N)
    #precision
    print (sum(precision) / len(precision) )
    
    slotid += 1



*** Slot no. =  1
922 users changed

###ALS IMPLICIT model


100%|████████████████████████████████████████████████████████████████████████████████| 20.0/20 [00:00<00:00, 45.89it/s]


Precision at rank  30
0.0

####Bayesian Personalised Ranking model


100%|██████████████████████████████████████████████████| 20/20 [00:00<00:00, 73.73it/s, correct=52.43%, skipped=19.84%]


Precision at rank  30
0.0
*** Slot no. =  2
950 users changed

###ALS IMPLICIT model


...

Precision at rank  30
0.0
*** Slot no. =  4
973 users changed

###ALS IMPLICIT model


100%|████████████████████████████████████████████████████████████████████████████████| 20.0/20 [00:00<00:00, 29.62it/s]


Precision at rank  30
0.0

####Bayesian Personalised Ranking model


100%|██████████████████████████████████████████████████| 20/20 [00:00<00:00, 53.33it/s, correct=53.04%, skipped=21.21%]


Precision at rank  30
0.0
